In [7]:
import re
from tabula import read_pdf
import PyPDF2
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 📌 경로 및 설정
pdf_path = "data/tax_etc/연말정산_Q&A.pdf"
COLLECTION_NAME = "tax1"
PERSIST_DIRECTORY = "tax1"


# 📌 페이지 번호 및 【】 형식 제거 함수
def clean_page_numbers_and_brackets(text):

    #  페이지 번호 패턴 제거 (- 숫자 - 형태)
    text = re.sub(r'^- \d{1,3} -', '', text, flags=re.MULTILINE)
    
    # 【】로 둘러싸인 텍스트 제거
    text = re.sub(r'【.*?】', '', text, flags=re.MULTILINE)
    
    return text.strip()



# 📌 임베딩 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 📌 모든 페이지 처리
documents = []

# 📄 **텍스트 추출 및 확인
try:
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        
        # 23페이지부터 시작
        for page_num in range(23, num_pages):
            page = reader.pages[page_num]
            page_text = page.extract_text() or ""
            cleaned_text = clean_page_numbers_and_brackets(page_text)
            
            # if cleaned_text:
            #     # 확인용 출력
            #     print(f"\n📄 [Page {page_num + 1}] Extracted Text:")
            #     print(cleaned_text[:300])  # 처음 300자만 출력
                
            #     documents.append(Document(
            #         page_content=cleaned_text,
            #         metadata={
            #             "source": pdf_path,
            #             "type": "text",
            #             "page": page_num + 1
            #         }
            #     ))
except Exception as e:
    print(f"⚠️ Text Extraction Error: {e}")

try:
    tables = read_pdf(
        pdf_path,
        pages="23-",
        multiple_tables=True,
        lattice=True,  # 격자형 표 인식
        guess=True     # 표 자동 인식 최적화
    )
    if tables:
        for i, table in enumerate(tables):
            table_text = table.to_string(index=False, header=True)
            # if table_text.strip():  # 빈 문자열 방지
            #     print(f"\n📊 [Table {i + 1}] Extracted Table:")
            #     print(table_text[:300])  # 처음 300자만 출력

            #     documents.append(Document(
            #         page_content=table_text,
            #         metadata={
            #             "source": pdf_path,
            #             "type": "table",
            #             "page": 23,
            #             "table_index": i + 1
            #         }
            #     ))
    else:
        print("⚠️ No tables were detected on the specified pages.")
except Exception as e:
    print(f"⚠️ Table Extraction Error: {e}")


In [2]:
# 📌 **텍스트 분할 (Splitting)** 🔥
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 하나의 청크에 들어갈 최대 문자 수
    chunk_overlap=150  # 청크 간 겹치는 문자 수
)

# 🔄 문서 분할
split_documents = text_splitter.split_documents(documents)


In [3]:

# 📌 Chroma 벡터 스토어에 저장
# if documents:
#     vector_store = Chroma.from_documents(
#         documents=documents,
#         embedding=embedding_model,
#         collection_name=COLLECTION_NAME,
#         persist_directory=PERSIST_DIRECTORY
#     )
#     print(f"✅ Successfully stored {len(documents)} documents in the vector store.")
# else:
#     print("⚠️ No documents to store in the vector database.")

vector_store = Chroma.from_documents(
        documents=split_documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )




In [4]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10}
)


In [5]:

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        #추가한 부분#
        사용자의 질문이 정확한 법령명이나 조항을 다루고 있지 않더라도, 질문의 맥락과 키워드를 분석하여 가장 가까운 관련 법령 및 조항을 찾아 답변하세요.
        법령명에서 법과 같은 접미어가 생략된 경우에도 동일한 의미로 간주하세요.
        법령명과 조항 번호가 다소 부정확하게 입력되었더라도, AI가 가능한 한 사용자의 의도를 파악하여 올바른 법령과 조항으로 연결하세요.
        질문에 특정 조항(예: 제1조)이 포함된 경우, 해당 조항의 제목, 본문, 연혁, 주석을 종합적으로 확인해 답변하세요.
        
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser



In [6]:
# chain.invoke("사업소득이 4천만원 이하일때 소상공인 공제부금 소득공제의 한도를 알려줘")
# chain.invoke("소상공인 공제부금 소득공제의 한도는 얼마야? 사업소득이 1억을 넘어갔을 때")
# chain.invoke("근로소득 산출세액이 130만원 이하일 때 세액공제금액을 알려줘")
chain.invoke("어린이집 입소료는 교육비 세액공제 대상에 포함되어있어?")
# chain.invoke("근로제공 기간동안 지출한 비용에 대해서만 공제가능한 항목에 대해서 알려줘")



'아니요, 어린이집 입소료는 교육비 세액공제 대상에 포함되지 않습니다. 어린이집에 지출한 교육비 중에서 교육비 세액공제 대상에 포함되는 항목은 「영유아보육법」 제38조에 따라 정해진 보육료와 특별활동비(도서 구입비 포함, 재료비 제외)입니다. 따라서 입소료, 현장학습비, 차량 운행비 등은 실비 성격의 기타 필요 경비로 간주되어 교육비 공제 대상에 해당하지 않습니다.'